In [134]:
import numpy as np
from typing import List, Tuple
from math import gcd
import math
import random
import logging

def prime_mod_inv(x, mod):
    return pow(x, mod - 2, mod)


def modinv(x,p):
    # Computes the moduler inversion of x ** p-2 mod p,
    # for p prime
    return pow(x,p-2,p)

In [61]:

# Sieve of Eratosthenes
# Code by David Eppstein, UC Irvine, 28 Feb 2002
# http://code.activestate.com/recipes/117119/

def primes_generator():
    """ Generate an infinite sequence of prime numbers.
    """
    # Maps composites to primes witnessing their compositeness.
    # This is memory efficient, as the sieve is not "run forward"
    # indefinitely, but only as long as required by the current
    # number being tested.
    #
    D = {}
    
    # The running integer that's checked for primeness
    q = 2
    
    while True:
        if q not in D:
            # q is a new prime.
            # Yield it and mark its first multiple that isn't
            # already marked in previous iterations
            # 
            yield q
            D[q * q] = [q]
        else:
            # q is composite. D[q] is the list of primes that
            # divide it. Since we've reached q, we no longer
            # need it in the map, but we'll mark the next 
            # multiples of its witnesses to prepare for larger
            # numbers
            # 
            for p in D[q]:
                D.setdefault(p + q, []).append(p)
            del D[q]
        
        q += 1

In [3]:
def generate_primes(n: int):
    primes = []
    i = 0
    for p in primes_generator():
        i += 1
        primes.append(p)
        if i == n:
            return primes

In [109]:
%%time
BIG_PRIMES_CACHE = generate_primes(1_000_000)[990_000:]

CPU times: user 9.49 s, sys: 220 ms, total: 9.71 s
Wall time: 9.71 s


In [110]:
BIG_PRIMES_CACHE[0]

15318911

# Задание 1

1) Реализуйте алгоритмы шифрования и дешифрования, а также генерации ключей для системы Эль - Гамаля. 

2) Зашифруйте и расшифруйте несколько сообщений с разными параметрами и длинами сообщений, составьте таблицу показывающую скорость работы криптосистемы. 

3) Покажите корректность криптосистемы.

In [113]:
def generate_keys(primes_db: List[int] = BIG_PRIMES_CACHE) -> Tuple[Tuple[int], int]:
    p = int(np.random.choice(primes_db))
    g = random.randint(2, p)
    
    x = random.randint(pow(10, 20), pow(10, 30)) #gen_key(p)#

    h = pow(g, x, p)
    
    public_key = (p, g, h)
    priate_key = x
    
    return public_key, priate_key

In [114]:
generate_keys()

((15477811, 4340224, 1154316), 158806008710649360579198454631)

In [115]:
def encrypt(message: List[int], pub_key: Tuple[int]) -> Tuple[List[int], int]:
    (p, g, h) = pub_key
    k = random.randint(2, p)
    K = pow(h, k, p)
    
    code = [(m*K) % p for m in message]
    O_sk = pow(g, k, p)
    
    return code, O_sk

     
def decrypt(code: List[int], pub_key: Tuple[int], O_sk: int, private_key: Tuple[int]) -> List[int]:
    (p, g, h) = pub_key
    x = private_key
    K = pow(O_sk, x, p)
    K_invert = modinv(K, p)
    message = [(c*K_invert)%p for c in code]
    return message

In [116]:
pub_key, private_key = generate_keys()
message = [55, 66, 89, 78, 93, 22]
code, O_sk = encrypt(message, pub_key)
mesage_restored = decrypt(code, pub_key, O_sk, private_key)

message == mesage_restored

True

In [117]:
message

[55, 66, 89, 78, 93, 22]

In [118]:
mesage_restored

[55, 66, 89, 78, 93, 22]

# Задание 2

Реализовать алгоритмы дискретного логарифмирования: **Полный перебор степеней** и **Алгоритм больших и малых шагов**. 
Протестировать их на различных примерах. \
Детально сравнить скорость их работы.

a^x=b (mod p) \
h = g^x mod p

### Полный перебор степеней

In [130]:
p = int(np.random.choice(BIG_PRIMES_CACHE))
g = random.randint(2, p)

x = random.randint(pow(10, 10), pow(10, 20)) #gen_key(p)#

h = pow(g, x, p)

(p, g, h), x

((15455183, 10250396, 9900931), 3708557128006953853)

In [131]:
def brute_force(h, g, p) -> int:
    for x in range(1, p):
        if h == pow(g, x, p):
            return x

In [132]:
%%time
x_ = brute_force(h, g, p)

CPU times: user 4.76 s, sys: 0 ns, total: 4.76 s
Wall time: 4.76 s


In [133]:
pow(g, x_, p),  h

(9900931, 9900931)

###  Алгоритм больших и малых шагов

In [140]:
def bsgs(h, g, p):
    '''
    Solve for x in h = g^x mod p given a prime p.
    If p is not prime, you shouldn't use BSGS anyway.
    '''
    N = math.ceil(math.sqrt(p - 1))  # phi(p) is p-1 if p is prime

    # Store hashmap of g^{1...m} (mod p). Baby step.
    tbl = {pow(g, i, p): i for i in range(N)}

    # Precompute via Fermat's Little Theorem
    c = pow(g, N * (p - 2), p)

    # Search for an equivalence in the table. Giant step.
    for j in range(N):
        y = (h * pow(c, j, p)) % p
        if y in tbl:
            return j * N + tbl[y]

    # Solution not found
    return None

In [141]:
%%time
x_ = bsgs(h, g, p)

CPU times: user 6.34 ms, sys: 0 ns, total: 6.34 ms
Wall time: 6.12 ms


In [142]:
pow(g, x_, p),  h

(9900931, 9900931)

# Задание 3

Расшифровать шифротекст Эль - Гамаля без закрытого ключа, согласно вариантам.\
Для этого найдите закрытый ключ с помощью алгоритмов дискретного логарифмирования.

 5 Вариант \
 **P, g, h, O_sk, m** \
 а) (1621, 2, 8, 64, 1374);\
 б) (401132107, 5, 349975032, 262582374, 17960572);

In [150]:
p, g, h, O_sk, m = (1621, 2, 8, 64, 1374)

public_key = (p, g, h)

x = bsgs(h, g, p)

message = decrypt([m], pub_key, O_sk, x)

message, x

([5035363], 3)

In [151]:
p, g, h, O_sk, m = (401132107, 5, 349975032, 262582374, 17960572)

public_key = (p, g, h)

x = bsgs(h, g, p)

message = decrypt([m], pub_key, O_sk, x)

message, x

([14413052], 205549142)

In [154]:
pub_key, private_key = generate_keys()
message = [55, 66, 89, 78, 93, 22]
code, O_sk = encrypt(message, pub_key)

(p, g, h) = pub_key
x_ = bsgs(h, g, p)

mesage_restored = decrypt(code, pub_key, O_sk, x_)

message == mesage_restored


True

# Задание 4

Расшифруйте без закрытого ключа следующее сообщение.  \
**P, g, h, O_sk, m** \

In [156]:
def restore_no_private(code: List[int], pub_key: Tuple[int], O_sk: int) -> int:
    (p, g, h) = public_key
    x = bsgs(h, g, p)
    return decrypt(code, public_key, O_sk, x)

In [157]:
%%time
p = 260147055818248197894653524697;
g = 81845634689728340040520901679;
h = 202356722887630660067216789213;
O_sk = 5306753730871089882973367602;
m = 135691234701387372733541060211;

public_key = (p, g, h)
restore_no_private(m, public_key, O_sk)

KeyboardInterrupt: 

In [ ]:
%%time
p = 20877322939496718999507886320561262006149588197137;
g = 870843123424618675163716010659673726076476758909;
h = 7207317984241552030871178883064350738879910571494;
O_sk = 6796864572822288687416809010479769477246880690656;
m = 282675013090798290915650697860421629786835997398;

public_key = (p, g, h)
restore_no_private(m, public_key, O_sk)

In [ ]:
%%time
p = 2876533531729072512082818329952287785554819082407936795733805489728591;
g = 2533057351760799307616363212416778342469125538911820746531544613426542;
h = 1050801438014966439216366236279672394903289313289249053678609934452775;
O_sk = 2242230984295454322539052136800744886060744987604416480499149266658790;
m = 1371849664498900949800837594872913496467147428907280108161021265852474;

public_key = (p, g, h)
restore_no_private(m, public_key, O_sk)

In [ ]:
%%time
p = 52854404059991509314069937557578892569539085858940987255601864390261570465112144173788144130257527;
g = 4853368270202563286252009257092194687539098185568841022007075472586039265525482056853510150449122;
h = 52854404059991509314069937557578892569539085858940987255601864390261570465112144173788144130257527;
O_sk = 3890686571444456836554231821236512191047789612172394855114365344076687887158571773618974369355001;
m = 37480164404707175984456519081463765320456291484151561998847734775738831885826034565181167830921699;

public_key = (p, g, h)
restore_no_private(m, public_key, O_sk)